<a href="https://colab.research.google.com/github/gwang111/MachineLearningAndOptimization/blob/main/hw1/Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sklearn

In [ ]:
# imports
import pandas as pd

# For Question 2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# For Question 3
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score

1.

The code block below:

1.   Reads in the spam.csv from the most immediate directory. Making sure the coding is for English
2.   Next we select all the rows and the first two columns and assign that as the dataframe
3.  Next replace all instances of ham in the df as 0 and then replace all instances of spam in the df as 1
4.  Next rename the columns v1 to label and v2 to message
5.  Finally drop duplicate rows and then reset the indexes accounting for the dropped rows


In [ ]:
df = pd.read_csv('spam.csv', encoding='latin-1')
df = df.iloc[:, :2]
df = df.replace('ham', 0).replace('spam', 1)
df = df.rename(columns = {'v1': 'label', 'v2': 'message'})
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5164,1,This is the 2nd time we have tried 2 contact u...
5165,0,Will Ì_ b going to esplanade fr home?
5166,0,"Pity, * was in mood for that. So...any other s..."
5167,0,The guy did some bitching but I acted like i'd...


2.

**Tokenizing data:** Given theses string, the act of tokenizing is to split the inputs up into keywords, elements, etc that are referred to as tokens.

**CountVectorizer:** For CountVectorizer, using the default token_pattern input, 2 or more alphanumeric characters in a row are considered a token.

In [ ]:
messages = df['message']
y = df['label']
x = CountVectorizer().fit_transform(messages)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2021)
x

<5169x8672 sparse matrix of type '<class 'numpy.int64'>'
	with 68018 stored elements in Compressed Sparse Row format>

**Unique Tokens Identified**: 4767
This was found by summing up all the columns and then counting how many occurances of 1 occurred

In [ ]:
token_occ = x.toarray().sum(axis=0)

print(token_occ.shape)

unique = 0
for i in token_occ:
    if i == 1:
        unique = unique + 1

# unique words
print(unique)

(8672,)
4767


Analyzing the training data, the training set does not contain at least one instance of each token.

In [ ]:
train_occ = x_train.toarray().sum(axis=0)

print(train_occ.shape)

atleast_one = True
for i in train_occ:
    if i == 0: 
        atleast_one = False        
        break

print(atleast_one)

(8672,)
False


3.

**Accuracy**: All ML models in sklearn have a score method. This score method takes the model that we've trained using the x and y training data and then when score is called with x and y test passed in, it calculates how accurate the model was in predicting y from x. Accuracy is calculated as:
`number of correct predictions / total predictions` (i.e. the percentage of samples that the model was able to successfully predict). This is an important metric because this gives us an immediate idea of how good our machine learning model at doing its job (which is to find spam). For example, if the model correctly predicts 90% of the test set, we could potentially say that our model will give us the correct spam classification 90% of the time (how well the model did at marking spam as spam and ham as). But to get a better in-depth understanding of how well our model did, we must use precision and recall to further assess this.

**Precision**: `tp / (tp + fp)` where `tp` is the number of true positives and `fp` is the number of false positives. In other words, precision assesses the percentage of text messages classified as spam that were actually spam. Precision is very important because it is not enough to correctly classify all spam samples as spam. It is equally important that ham samples are not incorrectly being classified as spam (negatively impacts customers when a filter is filtering out important and relevant text messages).

**Recall**: `tp / (tp + fn)` where `tp` is the number of true positives (spam as spam) and `fn` is the number of false negatives (spam as ham). In other words, recall assesses the percentage of actual spam text messages that were correctly classified as spam. Recall is very important because we want to find all spam text messages (negatively impacts customers if spam text messages get through the filter).

In [ ]:
mnb_clf = MultinomialNB()
mnb_clf.fit(x_train, y_train)
y_hat = mnb_clf.predict(x_test)
[mnb_clf.score(x_test, y_test), precision_score(y_test, y_hat), recall_score(y_test, y_hat)]

[0.9777562862669246, 0.9154929577464789, 0.9219858156028369]

4.

In [ ]:
linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_hat = linear_svc.predict(x_test)
[linear_svc.score(x_test, y_test), precision_score(y_test, y_hat), recall_score(y_test, y_hat)]

[0.9816247582205029, 0.9919354838709677, 0.8723404255319149]

5.

In [ ]:
logistic_reg = LogisticRegression()
logistic_reg.fit(x_train, y_train)
y_hat = logistic_reg.predict(x_test)
[logistic_reg.score(x_test, y_test), precision_score(y_test, y_hat), recall_score(y_test, y_hat)]

[0.9787234042553191, 0.983739837398374, 0.8581560283687943]

6.

|| Accuracy | Precision | Recall |
| --- | --- | --- | --- |
| MultinomialNB | 0.9777562862669246 | 0.9154929577464789 | 0.9219858156028369 |
| LinearSVC | 0.9816247582205029 | 0.9919354838709677 | 0.8723404255319149 |
| LogisticRegression | 0.9787234042553191 | 0.983739837398374 | 0.8581560283687943 |

- I think what suprised me was the difference in Recall, namely 92% in MultinomialNB to 87% and 86% from LinearSVC and LogisticRegression respectively. (A 5% and 6% difference). With LinearSVC and LogisticRegression having lower recall but precision extremely high this ment that these two models were doing a great job at correctly assessing spam as spam but were classifying some spam samples as ham. This shows that LinearSVC and LogisticRegression seemed less sensative to the data. This would seem expected with LinearSVC because of an implemented thickness to counteract noise and overfitting but seems unexpected with LogisticRegression as LogisticRegression can be prone to overfitting.
- I think I would use MultinomialNB because for one, it performs well with NLP binary classification problems but also gives us more consistant percentages than LinearSVC and LogisticRegression with all 3 percentages in the 90s. Furthermore, while all 3 percentages are above 90%, it does a good job in this example of not being overly exact, this may be good when apply this model to various other text datasets as were are not overfitted as much as LinearSVC and LogisticRegression are to this spam.csv set. I'd rather my model be both able to correctly classify spam well and also be able to find all spam texts which MultinomialNB seems balance better than LinearSVC and LogisticRegression.

7.
- One caviate to consider is how informal the ham texts are (seemingly originating and collected from casual text conversations between familiar people). We also note how extremely formal and robotic the spam texts are consistantly like. With spam texts becoming more sophisticated, people formating them with more humanlike diction, using this dataset may not give us enough coverage on what is spam and what is ham.
- Another caviate is that some of these messages (specifcally spam messages) are extremely outdated (i.e messages referencing the year 2003, Nokia 3510i, 3G connection, etc.). These messages may not be relevant or apply to the year 2021 as diction and spam techniques have greatly changed since then.
- We must be wary that our model trained on this dataset may not be transferable when applied on text data that we have collected from 2021.